In [1]:
import findspark
findspark.find()

'/usr/local/opt/apache-spark/libexec'

In [2]:
from pyspark.sql import SparkSession
from sparknlp.pretrained import PretrainedPipeline
path = "hdfs://localhost:8020/todungnguyen/input/data_set_fusion.csv"
model = "hdfs://localhost:8020/todungnguyen/input/entity_recognizer_lg_fr"

In [3]:
# 4 executors, 1 core each
spark = SparkSession.builder \
    .appName("Spark NLP AGG")\
    .master("spark://todungnguyen:7077")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0")\
    .config("spark.executor.instances", "4")\
    .config("spark.executor.cores", "1")\
    .config("spark.jsl.settings.storage.cluster_tmp_dir", model)\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.2")\
    .getOrCreate()

In [4]:
df_spark = spark.read.option("header","true").csv(path)
df_spark.show(n=5)

+-------------+----------------+--------------------+--------------+------------+--------------------+--------------------+-------------------+
|type_de_carte| numero_de_carte|                iban|           nom|         tel|               email|             adresse|               date|
+-------------+----------------+--------------------+--------------+------------+--------------------+--------------------+-------------------+
|         Visa|4998333238637277|AL554284190202089...|  Ivan Mohamed|+33692972790|iDYjKeJPwUj.OfWk1...|6 Place du Pese L...|2018-10-05 03:03:48|
|         Visa|4532989368815505|XU224328033937652854| Ali Hernandez|+33907982343|1VHbkTUz+CUTb+un@...|1 Les Granges 164...|2020-04-02 04:17:10|
|         Visa|4916767269587027|DK636060640988463...|    Jack Gashi|+33348464665|NfdO++RnYVHTNKWI3...|2 Les Granges 164...|2020-09-10 01:39:24|
|         Visa|4485065267473279|BR243280562712302...|     Eric Tamm|+33451017552|gz6YlBr+vJrP@yaho...|10 Les Granges 16...|2018-08-22 11

In [5]:
data = df_spark.select('nom').toDF("text")
data.show(n=5)

+--------------+
|          text|
+--------------+
|  Ivan Mohamed|
| Ali Hernandez|
|    Jack Gashi|
|     Eric Tamm|
|Mohammed Kumar|
+--------------+
only showing top 5 rows



In [ ]:
## ERROR while do pipeline.transfrom(data): You are trying to use cluster mode without a proper shared filesystem.
pipeline_online = PretrainedPipeline.from_disk('entity_recognizer_lg_fr')

In [ ]:
## ERROR: load infini
# pipeline = PretrainedPipeline('entity_recognizer_lg', lang = 'fr')

In [6]:
%%time
## ERROR: HDFS
pipeline = PretrainedPipeline.from_disk(model)

IllegalArgumentException: Wrong FS: hdfs://localhost:8020/todungnguyen/input/entity_recognizer_lg_fr/cfb1e5c6e6fe_cdx, expected: file:///

In [ ]:
annotations = pipeline.transform(data)
annotations.show(n=5)

In [ ]:
def get_entity_agg(column):
    df_spark = spark.read.option("header","true").csv(path)
    data = df_spark.select(column).toDF("text")
    annotations = pipeline.transform(data)
    result = annotations.select(F.col("text"), F.explode("ner.result").alias("entity"))

    result = result.withColumn('ent', F.when(result['entity'] != "O", F.split(result['entity'], '-').getItem(1)).otherwise(result['entity'])) \
                    .drop("entity") \
                    .groupby("text", "ent") \
                    .agg(F.count("ent").alias("count"))

    w2 = Window.partitionBy("text").orderBy(F.col("count").desc())

    result = result.withColumn("row",F.row_number().over(w2)) \
                    .filter(F.col("row") == 1) \
                    .drop("row").orderBy("text") \
                    .groupby("ent").count(). \
                    orderBy(F.col("count").desc())
    
    return result.first()[0]